In [1]:
import numpy as np
from tensorflow.keras import optimizers,Sequential, layers, Model
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# 一维卷积后，再全连接获得三层预测
class FcCvModelReFCDepth(Model):
    def __init__(self):
        super(FcCvModelReFCDepth, self).__init__()
        # 多层全链接的第一层，每一道数据的提取
        # 实际上是在每道上的滑动全连接
        self.fcOne = Sequential([
            layers.Dense(64),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Dense(8),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        # 1.减少过拟合的情况
        # 2.模拟道数据的丢失
        self.dropout_fc = layers.Dropout(0.3)

        # 1维卷积综合提取特征
        self.conv1 = Sequential([
            layers.Conv1D(filters=64, kernel_size=9, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c1 = layers.Dropout(0.3)
        self.conv2 = Sequential([
            layers.Conv1D(filters=64, kernel_size=9, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c2 = layers.Dropout(0.3)
        # 1维卷积综合提取特征+降维
        self.conv3 = Sequential([
            layers.Conv1D(filters=128, kernel_size=4, strides=4, padding='valid'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c3 = layers.Dropout(0.3)
        # 1维卷积综合提取特征
        self.conv4 = Sequential([
            layers.Conv1D(filters=128, kernel_size=7, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c4 = layers.Dropout(0.3)
        self.conv5 = Sequential([
            layers.Conv1D(filters=128, kernel_size=7, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        self.dropout_c5 = layers.Dropout(0.3)
        # 1维卷积综合提取特征+降维
        self.conv6 = Sequential([
            layers.Conv1D(filters=256, kernel_size=4, strides=4, padding='valid'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
#         self.dropout_c6 = layers.Dropout(0.3)
        # 最终作出的预测
        # 深度预测
        self.fcThree_d1 = Sequential([
            layers.Dense(256)
        ])

        self.fcThree_d2 = Sequential([
            layers.Dense(256)
        ])

        self.fcThree_d3 = Sequential([
            layers.Dense(256)
        ])

    def call(self, inputs, training=None, mask=None):
        batchSize, dao_num, data_num = inputs.shape
        x = tf.reshape(inputs, (-1, 1024))
        x = self.fcOne(x)
        x = tf.reshape(x, (batchSize, dao_num, -1))
        x = tf.transpose(x, perm=[0, 2, 1])
        x = self.dropout_fc(x, training=training)
        x = self.conv1(x)
        x = self.dropout_c1(x, training=training)
        x = self.conv2(x)
        x = self.dropout_c2(x, training=training)
        x = self.conv3(x)
        x = self.dropout_c3(x, training=training)
        x = self.conv4(x)
        x = self.dropout_c4(x, training=training)
        x = self.conv5(x)
        x = self.dropout_c5(x, training=training)
        x = self.conv6(x)
#         x = self.dropout_c6(x, training=training)
        x = tf.reshape(x, (batchSize, -1))
        x_d1 = self.fcThree_d1(x)
        x_d2 = self.fcThree_d2(x)
        x_d3 = self.fcThree_d3(x)
        x_d = tf.concat([tf.expand_dims(x_d1,axis=1),tf.expand_dims(x_d2,axis=1),tf.expand_dims(x_d3,axis=1)],axis=1)
        return x_d

In [3]:
loss_log_path = './tf_dir/loss_all_layer3_fc_depth_4'
loss_summary_writer = tf.summary.create_file_writer(loss_log_path)

In [4]:
train_loss_depth_m = tf.keras.metrics.Mean('train_loss_depth', dtype=tf.float32)
dev_loss_depth_m = tf.keras.metrics.Mean('dev_loss_depth', dtype=tf.float32)

In [5]:
# model.load_weights('weights/weights_'+str(3000))

In [6]:
x = np.load('/openbayes/input/input0/data_all.npy')
# y = np.load('/openbayes/input/input1/label_time.npy')
# y_class = np.load('/openbayes/input/input2/label_class.npy')
y_depth = np.load('/openbayes/input/input2/label_depth.npy')*4
# y_class_oh = tf.one_hot(y_class.astype(np.int32),depth=8,axis=-1)

In [7]:
optimizer = optimizers.Adam(lr=1e-4)
epochs = 10000
model = FcCvModelReFCDepth()

In [8]:
# y = y * 1000

In [ ]:
for e in range(epochs):
    for i in range(12):
#         for step in range(2):[step*256:(step+1)*256]
        with tf.GradientTape() as tape:
            y_depth_pred = model(x[i*512:(i+1)*512],training=True)
            train_loss_depth = tf.reduce_mean(tf.losses.MSE(y_depth_pred, y_depth[i*512:(i+1)*512]))
        grads = tape.gradient(train_loss_depth, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_loss_depth_m(train_loss_depth)
    with loss_summary_writer.as_default():
        tf.summary.scalar('train_loss_depth', train_loss_depth_m.result(), step=e)
    train_loss_depth_m.reset_states()
    if e%100 == 0:
        for i in range(4):
            y_dev_depth_pred = model(x[(i+12)*512:(i+16)*512],training=False)
            dev_loss_depth = tf.reduce_mean(tf.losses.MSE(y_dev_depth_pred, y_depth[(i+12)*512:(i+16)*512]))
            dev_loss_depth_m(dev_loss_depth)
        with loss_summary_writer.as_default():
            tf.summary.scalar('dev_loss_depth', dev_loss_depth_m.result(), step=e)
        dev_loss_depth_m.reset_states()
        model.save_weights('weights_fc_depth_4/weights_'+str(e+1))